In [1]:
using LinearAlgebra

function cross_prod_matrix( k )
    return [ 0  -k[3] k[2];  
            k[3] 0   -k[1]; 
           -k[2] k[1] 0 ]
end 

cross_prod_matrix (generic function with 1 method)

In [2]:
function rot_matrix_3d_transform( k, theta )
    mat0 = [ 1 0          0; 
             0 cos(theta) -sin(theta); 
             0 sin(theta) cos( theta )]
    
    if isapprox( k[1], 0.0 ) && isapprox( k[2], 0.0 )
        u = [ 0.0, 0.0, 1.0 ]
    else 
        u = [ k[2], -k[1], 0 ]
        u /= norm( u )
    end 
    
    v = cross( k, u )
    trans = hcat( k, u, v )
    return trans*mat0*transpose( trans )
end 

rot_matrix_3d_transform (generic function with 1 method)

In [3]:
function rot_matrix_3d( k, theta )
    
    K = cross_prod_matrix( k )
    return I + sin(theta)K + (1-cos(theta))K^2
end 

rot_matrix_3d (generic function with 1 method)

In [4]:
k = [ rand()- .5 for x in 1:3 ]
k = k/norm(k)
theta = 2*pi*rand()
k

3-element Vector{Float64}:
  0.8121952090018213
 -0.25989258345431376
 -0.5222976043788928

In [15]:
@time R1 = rot_matrix_3d_transform( k, theta )

  0.000030 seconds (31 allocations: 1.781 KiB)


3×3 Matrix{Float64}:
  0.998305   -0.0531189  0.0237953
  0.051016    0.995355   0.0816433
 -0.0280216  -0.0802909  0.996378

In [26]:
@time R2 = rot_matrix_3d( k, theta )
@assert isapprox( R1, R2 )
R2

  0.000013 seconds (28 allocations: 1.719 KiB)


3×3 Matrix{Float64}:
  0.998305   -0.0531189  0.0237953
  0.051016    0.995355   0.0816433
 -0.0280216  -0.0802909  0.996378

In [17]:
function params_rotation_3d( R )
   
    theta = acos( (R[1,1] + R[2,2] + R[3,3] -1 )/2 )
    k1 = (R[3,2]-R[2,3])/(2*sin(theta))
    k2 = (R[1,3]-R[3,1])/(2*sin(theta))
    k3 = (R[2,1]-R[1,2])/(2*sin(theta))
    
    return [k1, k2, k3], theta
end

params_rotation_3d (generic function with 1 method)

In [19]:
@time kk, th = params_rotation_3d( R1 )


  0.000017 seconds (8 allocations: 272 bytes)


([-0.8121952090018223, 0.2598925834543141, 0.5222976043788935], 0.09985507593074786)

In [20]:
params_rotation_3d( R2 )

([-0.8121952090018222, 0.25989258345431404, 0.5222976043788934], 0.09985507593074786)

In [30]:
#id = Matrix( I, 3, 3 )
@time det( R1-I )

  0.000012 seconds (4 allocations: 352 bytes)


8.357819026304529e-19